[View in Colaboratory](https://colab.research.google.com/github/MarcinWylot/PredictingReorders/blob/master/PredictingReorders.ipynb)

**GOAL**: Build a model that predicts when given product will be ordered again.

**DATASET**: [Online Retail Data Set ](https://archive.ics.uci.edu/ml/datasets/online+retail)

First we need to download our dataset, I have it on my google drive. 

In [1]:

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#download my file
!rm retail_data*
downloaded = drive.CreateFile({'id': '1bgr8z7CGeW8pwvKV8kFLFumMFqRcmhFg'})
downloaded.GetContentFile('./retail_data_2.zip')
!unzip retail_data_2.zip
!ls -hl

Archive:  retail_data_2.zip
  inflating: retail_data.csv         
total 52M
drwxr-xr-x 1 root root 4.0K Jul 17 08:59 datalab
-rw-r--r-- 1 root root 6.9M Jul 17 14:30 retail_data_2.zip
-rw-rw-r-- 1 root root  45M May 14 10:47 retail_data.csv


In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          40G  5.2G   32G  14% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G     0  6.4G   0% /opt/bin
/dev/sda1        46G  5.9G   40G  14% /etc/hosts
shm              64M     0   64M   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


Let's load data to DataFrame and see what we have there

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 


data = pd.read_csv('./retail_data.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,1,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,5504.0,United Kingdom
1,1,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom
2,1,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,5504.0,United Kingdom
3,1,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom
4,1,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom


let's check is we're dafe with NaNs

In [4]:
data.isnull().any(axis=0)

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool

# Feature Engineering 

Let's see how hot are the items, a quick values count  adding  frequencies

In [5]:
data['freq'] = data.groupby('StockCode')['StockCode'].transform('count')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,freq
0,1,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,5504.0,United Kingdom,2313
1,1,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom,355
2,1,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,5504.0,United Kingdom,296
3,1,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom,474
4,1,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom,452


Ok, so now let's calculate the time interval from the last order of a particulat item

this is essentially what we will try to predict, in what time interval from last order for a partucilat product we will see it ordered again 

additionally we clean a bit the data, negative time intervals are set to -1

In [6]:
import datetime

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

data['time_from_last_order'] =\
  data.sort_values(['StockCode','InvoiceDate']).groupby('StockCode').InvoiceDate\
  .diff()\
  .apply(lambda x: x.seconds if x >= pd.Timedelta(0) else -1.0)
data.sort_values(['StockCode','InvoiceDate']).head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,freq,time_from_last_order
31,6,10002,INFLATABLE POLITICAL GLOBE,48,2010-12-01 08:45:00,0.85,237.0,France,73,-1.0
142,18,10002,INFLATABLE POLITICAL GLOBE,12,2010-12-01 09:45:00,0.85,3752.0,United Kingdom,73,3600.0
4272,392,10002,INFLATABLE POLITICAL GLOBE,1,2010-12-02 14:23:00,0.85,NaN,United Kingdom,73,16680.0
5466,499,10002,INFLATABLE POLITICAL GLOBE,1,2010-12-03 11:19:00,0.85,5621.0,United Kingdom,73,75360.0
5546,501,10002,INFLATABLE POLITICAL GLOBE,5,2010-12-03 11:28:00,1.66,NaN,United Kingdom,73,540.0


Time between orders in whcih an item occurs, we take into accont past 3 orders. 
This is to capdute dynamicity of orders for a product.

These features will give us a picture how often the item recently apeared in baskets (just 3 last orders)

In [7]:
data['time_from_last_order_t-1'] = data.sort_values(['StockCode','InvoiceDate']).groupby('StockCode').time_from_last_order.shift(1).apply(lambda x: x if x >= 0 else -1.0)
data['time_from_last_order_t-2'] = data.sort_values(['StockCode','InvoiceDate']).groupby('StockCode').time_from_last_order.shift(2).apply(lambda x: x if x >= 0 else -1.0)
data['time_from_last_order_t-3'] = data.sort_values(['StockCode','InvoiceDate']).groupby('StockCode').time_from_last_order.shift(3).apply(lambda x: x if x >= 0 else -1.0)
data.isnull().any(axis=0)

InvoiceNo                   False
StockCode                   False
Description                  True
Quantity                    False
InvoiceDate                 False
UnitPrice                   False
CustomerID                   True
Country                     False
freq                        False
time_from_last_order        False
time_from_last_order_t-1    False
time_from_last_order_t-2    False
time_from_last_order_t-3    False
dtype: bool

Now we add some stats around products and time elapesd from their last orders

max is easy to aggragate 

for others we want to take into account only positive time deltas, as there can be some NaT or -1 which we use to mark the first time orders 

In [8]:
sts = data.groupby('StockCode').time_from_last_order.max().to_frame()
sts.columns = ['time_from_last_order_max']
sts['time_from_last_order_min'] = data.groupby('StockCode').time_from_last_order.agg([lambda x: x[x >= 0].min()])
sts['time_from_last_order_median'] = data.groupby('StockCode').time_from_last_order.agg([lambda x: x[x >= 0].median()])
sts['time_from_last_order_mean'] = data.groupby('StockCode').time_from_last_order.agg([lambda x: x[x >= 0].mean()])
sts.fillna(-1.0,inplace=True)
sts.reset_index(inplace=True)
data = data.merge(sts, on='StockCode', how='left', copy=False)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,freq,time_from_last_order,time_from_last_order_t-1,time_from_last_order_t-2,time_from_last_order_t-3,time_from_last_order_max,time_from_last_order_min,time_from_last_order_median,time_from_last_order_mean
0,1,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,5504.0,United Kingdom,2313,-1.0,-1.0,-1.0,-1.0,84900.0,0.0,2700.0,11253.321799
1,1,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom,355,-1.0,-1.0,-1.0,-1.0,86040.0,0.0,11820.0,32725.423729
2,1,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,5504.0,United Kingdom,296,-1.0,-1.0,-1.0,-1.0,85800.0,0.0,12900.0,33462.508475
3,1,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom,474,-1.0,-1.0,-1.0,-1.0,86280.0,0.0,6480.0,24682.452431
4,1,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,5504.0,United Kingdom,452,-1.0,-1.0,-1.0,-1.0,86280.0,0.0,5460.0,21663.858093


In [9]:
data.isnull().any(axis=0)

InvoiceNo                      False
StockCode                      False
Description                     True
Quantity                       False
InvoiceDate                    False
UnitPrice                      False
CustomerID                      True
Country                        False
freq                           False
time_from_last_order           False
time_from_last_order_t-1       False
time_from_last_order_t-2       False
time_from_last_order_t-3       False
time_from_last_order_max       False
time_from_last_order_min       False
time_from_last_order_median    False
time_from_last_order_mean      False
dtype: bool

A chance to order a particular item can be depended on the day of week so let's add it to out features

In [0]:
data['day-of-week'] = data['InvoiceDate'].dt.dayofweek


We have some additional features around time, products, and orders

in total we will use the followint features for the model

numericat features: (to be normalized)
* Quantity 
* UnitPrice 
* freq
* time_from_last_ordert-1
* time_from_last_ordert-2
* time_from_last_ordert-3
* time_from_last_order_max
* time_from_last_order_min
* time_from_last_order_median
* time_from_last_order_mean

categorical features: (to be binarized)
* Country
* day-of-week

The output we well predict is **time_from_last_order**. It gives time that has elapsed from the last order to the next order, from this value we can easilly compute WHEN (date time) the product will be reordered

There is still room for more features developement, especially correlating user and basket/invoice, also calculateing probabilities of however for the sake of time I skip it a.t.m.

# Building a dataset for model training and validation

In [11]:
X=data[['Quantity','UnitPrice','Country','freq',
       'time_from_last_order_t-1', 'time_from_last_order_t-2',
       'time_from_last_order_t-3', 'time_from_last_order_max',
       'time_from_last_order_min', 'time_from_last_order_median',
       'time_from_last_order_mean', 'day-of-week']]

y=data['time_from_last_order']

print(X.isnull().any(axis=0))
print(y.isnull().any(axis=0))

Quantity                       False
UnitPrice                      False
Country                        False
freq                           False
time_from_last_order_t-1       False
time_from_last_order_t-2       False
time_from_last_order_t-3       False
time_from_last_order_max       False
time_from_last_order_min       False
time_from_last_order_median    False
time_from_last_order_mean      False
day-of-week                    False
dtype: bool
False


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, X.shape, y.shape)

(433527, 12) (108382, 12) (433527,) (108382,) (541909, 12) (541909,)


# Data transformation mapper
We need to standarize and binarize our features.
To make our life a bit easier we employ DataFrameMapper.

In [13]:
!pip install sklearn-pandas

from sklearn.preprocessing import StandardScaler, LabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn_pandas import DataFrameMapper

mapper = DataFrameMapper([
  (['Quantity','UnitPrice','freq',\
   'time_from_last_order_t-1', 'time_from_last_order_t-2', 'time_from_last_order_t-3',\
    'time_from_last_order_max', 'time_from_last_order_min','time_from_last_order_median','time_from_last_order_mean'],\
      StandardScaler()),
   ('day-of-week', LabelBinarizer()),
   ('Country', LabelBinarizer())
])

#x=mapper.fit_transform(X)
#x.shape

# Model based on NN (Keras)

In [14]:
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline

def nn_model():
  size_n = 54 #number of columns after DataMapper transformation
  # create model
  model = Sequential()
  model.add(Dense(size_n, input_dim=size_n, kernel_initializer='random_normal', activation='relu'))
  model.add(Dense(size_n, kernel_initializer='random_normal', activation='relu'))
  #model.add(Dense(size_n*2, kernel_initializer='random_normal', activation='relu'))
  #model.add(Dense(size_n*2, kernel_initializer='random_normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='random_normal'))
  # Compile model
  model.compile(loss='mean_absolute_error', optimizer='sgd')
  return model

reg =  KerasRegressor(build_fn=nn_model, epochs=5, batch_size=1000, verbose=1)

#pipeline.fit_transform(x_train, y_train["time_from_last_order"])

Using TensorFlow backend.


Pipeline allows us to combine various steps of the process, data preprocessing, model fitting, and with it we can later quickly add, change, move steps. Can be also usefull for some kind of grid search and evaluating multiple models. 

In [0]:
pipeline = Pipeline([('mapper', mapper),
                    ('reg', reg)
                   ])

No we are ready to fir our model....

In [16]:
pipeline.fit(X_train, y_train)

Epoch 1/5
433527/433527 [==============================] - 3s 6us/step - loss: 27927.8541
Epoch 2/5
433527/433527 [==============================] - 3s 6us/step - loss: 24798.7184
Epoch 3/5
433527/433527 [==============================] - 3s 6us/step - loss: 24660.4717
Epoch 4/5
433527/433527 [==============================] - 3s 6us/step - loss: 24333.1022
Epoch 5/5
433527/433527 [==============================] - 2s 6us/step - loss: 23409.2038


Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=False,
        features=[(['Quantity', 'UnitPrice', 'freq', 'time_from_last_order_t-1', 'time_from_last_order_t-2', 'time_from_last_order_t-3', 'time_from_last_order_max', 'time_from_last_order_min', 'time_from_last_order_median', 'time_from_la...lse, sparse=False)), ('reg', <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fb3caf97208>)])

...and evaluate it

In [17]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

print('Mean absolute error in hours: {}, test {}'.format(
    mean_absolute_error(y_train,y_train_pred)/60/60,
    mean_absolute_error(y_test,y_test_pred)/60/60 ) )

108382/108382 [==============================] - 0s 2us/step
Mean absolute error in hours: 6.408706203424617, test 6.446535329545071


# Model based on ensemble learning (XGBRegressor)
Additionally to a new model we will do some hyperpatameter optymization with RandomizedSearchCV. We cannot go crazy with nymber of iterations and params due to limited resurces.  

In [18]:
!pip install xgboost
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

import scipy.stats as st

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 20),
    "max_depth": st.randint(3, 20),
    "learning_rate": st.uniform(0.01, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}


xbst = XGBRegressor(nthread=-1)  

gs = RandomizedSearchCV(xbst, params, n_jobs=1, verbose=3, n_iter=2)  

Let's fit the model and see what params perform best.

In [19]:
gs.fit(mapper.fit_transform(X_train), y_train)  

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] colsample_bytree=0.9304906184484553, gamma=9.435773399569731, learning_rate=0.10379406529198455, max_depth=14, min_child_weight=8.34944550724054, n_estimators=19, reg_alpha=1.6182662690239484, subsample=0.9699975249214439 
[CV]  colsample_bytree=0.9304906184484553, gamma=9.435773399569731, learning_rate=0.10379406529198455, max_depth=14, min_child_weight=8.34944550724054, n_estimators=19, reg_alpha=1.6182662690239484, subsample=0.9699975249214439, score=0.19094689369256412, total=  36.8s
[CV] colsample_bytree=0.9304906184484553, gamma=9.435773399569731, learning_rate=0.10379406529198455, max_depth=14, min_child_weight=8.34944550724054, n_estimators=19, reg_alpha=1.6182662690239484, subsample=0.9699975249214439 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   38.3s remaining:    0.0s


[CV]  colsample_bytree=0.9304906184484553, gamma=9.435773399569731, learning_rate=0.10379406529198455, max_depth=14, min_child_weight=8.34944550724054, n_estimators=19, reg_alpha=1.6182662690239484, subsample=0.9699975249214439, score=0.19191216463680205, total=  36.8s
[CV] colsample_bytree=0.9304906184484553, gamma=9.435773399569731, learning_rate=0.10379406529198455, max_depth=14, min_child_weight=8.34944550724054, n_estimators=19, reg_alpha=1.6182662690239484, subsample=0.9699975249214439 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s


[CV]  colsample_bytree=0.9304906184484553, gamma=9.435773399569731, learning_rate=0.10379406529198455, max_depth=14, min_child_weight=8.34944550724054, n_estimators=19, reg_alpha=1.6182662690239484, subsample=0.9699975249214439, score=0.19309743797246792, total=  36.9s
[CV] colsample_bytree=0.8843608198660169, gamma=0.09997341401008053, learning_rate=0.0681555189578482, max_depth=10, min_child_weight=78.45226308228736, n_estimators=5, reg_alpha=36.711574019594494, subsample=0.9281415562553603 
[CV]  colsample_bytree=0.8843608198660169, gamma=0.09997341401008053, learning_rate=0.0681555189578482, max_depth=10, min_child_weight=78.45226308228736, n_estimators=5, reg_alpha=36.711574019594494, subsample=0.9281415562553603, score=-0.3133096097517989, total=   6.9s
[CV] colsample_bytree=0.8843608198660169, gamma=0.09997341401008053, learning_rate=0.0681555189578482, max_depth=10, min_child_weight=78.45226308228736, n_estimators=5, reg_alpha=36.711574019594494, subsample=0.9281415562553603 
[

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.3min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=2, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb3bc5158d0>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb3bc515748>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb3bc515908>, 'colsample_bytree...e35d68>, 'min_child_weight': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb3bee35d68>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          re

Finally, we evaluate the best estimator selected with the RandomizedSearchCV.

In [20]:
y_train_pred = gs.best_estimator_.predict(mapper.transform(X_train)) 
y_test_pred = gs.best_estimator_.predict(mapper.transform(X_test))

print('Mean absolute error in hours: {}, test {}'.format(
    mean_absolute_error(y_train,y_train_pred)/60/60,
    mean_absolute_error(y_test,y_test_pred)/60/60 ) )

Mean absolute error in hours: 5.83433174467624, test 6.621672564631657


# Simple Stacking

First, we need to prepare features for our second level. We just use predictions from the previous levels here.

In [21]:
X_train_stack1 = pipeline.predict(X_train)
X_train_stack2 = gs.best_estimator_.predict(mapper.transform(X_train))
X_train_stack = np.column_stack([X_train_stack1, X_train_stack2])

X_test_stack1 = pipeline.predict(X_test)
X_test_stack2 = gs.best_estimator_.predict(mapper.transform(X_test))
X_test_stack = np.column_stack([X_test_stack1, X_test_stack2])

print(X_train_stack.shape, X_test_stack.shape)

108382/108382 [==============================] - 0s 2us/step
(433527, 2) (108382, 2)


Typically there is no need for super complex models on the 2nd level, hence here we employ LinearRegression. We could also use just average or build a NN here. 

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

reg = LinearRegression()
pipe_stacking = Pipeline([('poly',PolynomialFeatures(degree=1)),
                    ('scl', StandardScaler()),
                    ('reg', reg)
                   ])

In [23]:
pipe_stacking.fit(X_train_stack, y_train)

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=True, interaction_only=False)), ('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [24]:
y_train_pred = pipe_stacking.predict(X_train_stack)
y_test_pred = pipe_stacking.predict(X_test_stack)

print('Mean absolute error in hours: {}, test {}'.format(
    mean_absolute_error(y_train,y_train_pred)/60/60,
    mean_absolute_error(y_test,y_test_pred)/60/60 ) )

Mean absolute error in hours: 5.193101637245491, test 6.539057709638957
